In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython import get_ipython
get_ipython().magic('reset -sf')
import gym
import torch
from torch import nn
import matplotlib.pyplot as plt
import torch.utils.data as data_utils
import time
import numpy as np
import torch.nn.functional as F

In [2]:
#creating policy neural network 
class PolicyNetwork(nn.Module):
    def __init__(self, h): # h --> number of hidden units
        super().__init__()
        self.fc1 = nn.Linear(80*80, h)
        self.fc2 = nn.Linear(h,1)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        return x
    

In [3]:
def preproc(I):
    I = I[35:195]
    I = I[::2,::2,0]
    I[I==144] = 0
    I[I==109] = 0
    I[I!=0] = 1
    return I.astype(np.float).ravel()

In [4]:
def disreward(r, gamma):
    dis_r = np.zeros(len(r))
    r_sum = 0
    for s in reversed(range(0,len(r))):
        if r[s]!=0: r_sum = 0
        r_sum = r_sum*gamma + r[s]
        dis_r[s] = r_sum
    return dis_r  
            
    

In [38]:
H = 200
batch_size = 10
learning_rate = 1e-4
gamma = 0.99
decay_rate = 0.99
render = False
policy_net = PolicyNetwork(200)
env = gym.make('Pong-v0')

[2019-03-08 16:06:11,071] Making new env: Pong-v0


In [39]:
from torch import optim
observation = env.reset()
obs = torch.tensor(preproc(observation)).float()
linear_optim = optim.Adam(policy_net.parameters(),lr = 0.1)

In [ ]:
i = 0
env.reset()
episodes = 0
logp = []
r = []
cost = torch.tensor(0.0).float()
cost.requires_grad = True
s_r = 0
r_list = []
while True:
    if render:
        env.render()
    p = policy_net(obs.float())
    action = 2 if np.random.uniform() < p else  3
    observation, reward, done, info = env.step(action)
    obs = torch.tensor(preproc(observation)).float()
    if action == 2:
        logp.append(torch.log(p))
    else:
        logp.append(torch.log(1-p))
    r.append(reward)
    s_r+=reward
    if done:
       # print('getting cost')
        episodes = episodes+1
        dr = disreward(r,gamma)
        dr = dr-np.mean(dr)
        dr = dr/np.std(dr)
        logp = torch.tensor(logp).float()
        dr = torch.tensor(dr).float()
        logp.requires_grad = True
        dr.requires_grad = True
        cost = cost-torch.sum(logp*dr)
       # print('r: ', r)
       # print('logp: ', logp)
        r_list.append(s_r)
        s_r = 0
        logp = []
        r = []
        if episodes % batch_size == 0:
            print('avg reward; ', np.mean(r_list))
            print('cost: ', cost)
            cost.backward(retain_graph=True)
            linear_optim.step()
            cost = torch.tensor(0.0).float()
            cost.requires_grad = True
            r_list = []
        observation = env.reset()
        obs = torch.tensor(preproc(observation)).float()
            
    i = i+1
print(episodes)


/home/akhil/anaconda3/envs/deeprlbootcamp/lib/python3.5/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


avg reward;  -20.6
cost:  tensor(0.9877, grad_fn=<SubBackward0>)
avg reward;  -19.5
cost:  tensor(1.7604, grad_fn=<SubBackward0>)
avg reward;  -20.6
cost:  tensor(2.5436, grad_fn=<SubBackward0>)
avg reward;  -20.7
cost:  tensor(2.0983, grad_fn=<SubBackward0>)
avg reward;  -20.5
cost:  tensor(1.5417, grad_fn=<SubBackward0>)


In [27]:
render = True
while True:
    if render:
        env.render()
    p = policy_net(obs.float())
    action = 2 if np.random.uniform() < p else  3
    observation, reward, done, info = env.step(action)
    obs = torch.tensor(preproc(observation)).float()
    if done:
        env.reset()

/home/akhil/anaconda3/envs/deeprlbootcamp/lib/python3.5/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


KeyboardInterrupt: 